<a href="https://colab.research.google.com/github/tugbargn/Python-Codes/blob/main/cloud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
from keras.preprocessing import image
from os.path import join
import matplotlib.pyplot as plt
from PIL import Image
import scipy.misc
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense


input_size = 300
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/swimseg/metadata.csv")
data_dir = '/content/drive/MyDrive/Colab Notebooks/swimseg/'

NO_OF_IMAGES = 1013
img_ids = []


for i in range(NO_OF_IMAGES):
    if len(str(i+1)) == 4:
        item = str(i+1)
    elif len(str(i+1)) == 3:
        item = '0' + str(i+1)
    elif len(str(i+1)) == 2:
        item = '00' + str(i+1)
    elif len(str(i+1)) == 1:
        item = '000' + str(i+1)
    img_ids.append(item)



def get_image_and_mask(img_id):

    my_image = data_dir +  'images/' + str(img_id) + '.png'
    my_GT = data_dir +  'GTmaps/' + str(img_id) + '_GT.png'
    img = image.load_img(my_image,
                         target_size=(input_size, input_size))
    img = image.img_to_array(img)
    mask = image.load_img(my_GT,
                          grayscale=True, target_size=(input_size, input_size))
    mask = image.img_to_array(mask)
    img, mask = img / 255., mask / 255.

    return img, mask

for img_id in img_ids:
    img, mask = get_image_and_mask(img_id)
    #print (['Processing input image for ', img_id])


x, y = img, mask

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid'),
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Epoch 1/5


ValueError: ignored